In [39]:
#importo las librerias necesarias

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.datasets import make_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_squared_error

In [40]:
#cargo el dataset de training y el de test

df_training = pd.read_csv('./../data/training_new_preprocesing.csv')
df_test = pd.read_csv('./../data/test_new_procesing.csv')
df_training[:3]

,price,carat,depth,table,x,y,z,xy,xz,table_depth,...,color/weight_J,clarity/weight_I1,clarity/weight_IF,clarity/weight_SI1,clarity/weight_SI2,clarity/weight_VS1,clarity/weight_VS2,clarity/weight_VVS1,clarity/weight_VVS2,volume
0,4268,1.21,62.4,58.0,6.83,6.79,4.25,1.005891,1.607059,0.929487,...,0.826446,0.0,0.0,0.0,0.0,0.0,0.826446,0.0,0.0,197.096725
1,3513,1.02,61.6,58.0,6.40,6.35,3.93,1.007874,1.628499,0.941558,...,0.980392,0.0,0.0,0.0,0.0,0.0,0.980392,0.0,0.0,159.715200
2,1792,0.77,62.3,58.0,5.86,5.80,3.63,1.010345,1.614325,0.930979,...,1.298701,0.0,0.0,0.0,0.0,0.0,1.298701,0.0,0.0,123.376440


In [41]:
# Features + target

X = df_training[['carat',
                    'depth',
                    'table',
                    'x',
                    'y',
                    'z',
                    'xy',
                    'xz',
                    'table_depth',
                    'carat/dimensons',
                    'cut/weight_Fair',
                    'cut/weight_Good',
                    'cut/weight_Ideal',
                    'cut/weight_Premium',
                    'cut/weight_Very Good',
                    'color/weight_D',
                    'color/weight_E',
                    'color/weight_F',
                    'color/weight_G',
                    'color/weight_H',
                    'color/weight_I',
                    'color/weight_J',
                    'clarity/weight_I1',
                    'clarity/weight_IF',
                    'clarity/weight_SI1',
                    'clarity/weight_SI2',
                    'clarity/weight_VS1',
                    'clarity/weight_VS2',
                    'clarity/weight_VVS1',
                    'clarity/weight_VVS2',
                    'volume']]
y = pd.get_dummies(df_training[['price']], drop_first=True)
print(X.shape,y.shape)

(40455, 31) (40455, 1)


In [42]:
# Train + test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [43]:
# Scaling - StandardScaler
scaler = RobustScaler()
scaled_data = scaler.fit_transform(X_train)
scaled_data_test = scaler.transform(X_test)
scaled_data_test

array([[ 0.484375  ,  0.66666667,  0.33333333, ...,  0.        ,
         0.        ,  0.45056277],
       [ 0.484375  ,  0.73333333,  0.        , ...,  0.99009901,
         0.        ,  0.45919957],
       [-0.625     , -0.93333333,  0.66666667, ...,  0.        ,
         0.        , -0.61810124],
       ...,
       [-0.296875  ,  1.13333333, -0.33333333, ...,  0.        ,
         0.        , -0.30118618],
       [ 1.296875  ,  0.06666667,  0.        , ...,  0.        ,
         0.65359477,  1.25746608],
       [-0.546875  , -0.13333333, -0.33333333, ...,  0.        ,
         0.        , -0.54901957]])

In [44]:
regressor =  RandomForestRegressor()

In [45]:
hyperparameters = regressor.get_params()
model = regressor.fit(X_train, y_train.values.ravel())

In [46]:
y_pred = model.predict(X_test)

print('Model:', regressor, '\n')
print('Model hyperparameters:', hyperparameters, '\n')
#print('Ground truth target:', y_test, '\n')
print('Predicted target:', y_pred, '\n')

Model: RandomForestRegressor() 

Model hyperparameters: {'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': None, 'max_features': 1.0, 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False} 

Predicted target: [ 4719.92  8969.04   902.79 ...  1780.32 13175.87   638.77] 



In [47]:
y_pred.shape

(8091,)

In [48]:
type(y_pred)

numpy.ndarray

In [49]:
r2_ = model.score(X_test, y_test)
r2_

0.9812595564443609

In [50]:
rmse = mean_squared_error(y_test, y_pred)**0.5
rmse

545.9600963037153

In [51]:
param_grid = {'n_estimators': [16, 32, 64, 128, 256, 512],
              'max_depth': [2, 4, 8, 16]}

In [52]:
grid_search = GridSearchCV(model,
                           param_grid,
                           cv=5,
                           verbose=3,
                           scoring='neg_root_mean_squared_error',
                           n_jobs=-1)

In [53]:
%%time

grid_search.fit(X,y)

print('\n')
print('Best hyperparameters: ', grid_search.best_params_, '\n')
print('Best score: ', -grid_search.best_score_, '\n')

Fitting 5 folds for each of 24 candidates, totalling 120 fits


C:\Users\lourdes.ruiz\miniconda3\envs\project_new_env\Lib\site-packages\sklearn\model_selection\_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)




Best hyperparameters:  {'max_depth': 16, 'n_estimators': 512} 

Best score:  643.498376299416 

CPU times: total: 4min 46s
Wall time: 34min 35s


In [54]:
best_model = grid_search.best_estimator_

In [55]:
y_test = best_model.predict(df_test)

In [56]:
df_pred = pd.DataFrame(data=y_test)
df_pred

,0
0,3135.539174
1,5439.309358
2,9161.407612
3,4146.326190
4,1696.852676
...,...
13480,1864.231212
13481,2437.458371
13482,2996.295887
13483,2178.529684


In [57]:
df_pred = df_pred.reset_index()
df_pred

,index,0
0,0,3135.539174
1,1,5439.309358
2,2,9161.407612
3,3,4146.326190
4,4,1696.852676
...,...,...
13480,13480,1864.231212
13481,13481,2437.458371
13482,13482,2996.295887
13483,13483,2178.529684


In [58]:
df_rename = df_pred.rename(columns={"index": "id", "o": "price"})
df_rename

,id,0
0,0,3135.539174
1,1,5439.309358
2,2,9161.407612
3,3,4146.326190
4,4,1696.852676
...,...,...
13480,13480,1864.231212
13481,13481,2437.458371
13482,13482,2996.295887
13483,13483,2178.529684


In [59]:
final_prediction = df_rename.rename(columns={df_rename.columns[1]: "price" })
final_prediction

,id,price
0,0,3135.539174
1,1,5439.309358
2,2,9161.407612
3,3,4146.326190
4,4,1696.852676
...,...,...
13480,13480,1864.231212
13481,13481,2437.458371
13482,13482,2996.295887
13483,13483,2178.529684


In [60]:
export = final_prediction.to_csv('./../data/prediction_RandomForrest.csv', index=False)

In [61]:
pd.read_csv('./../data/prediction_RandomForrest.csv').head()

,id,price
0,0,3135.539174
1,1,5439.309358
2,2,9161.407612
3,3,4146.326190
4,4,1696.852676
